In [2]:
#%%
print("ok")

ok


In [3]:
allNormalText = open(r"./data/data-normal.txt", 'r', encoding='utf-8').read()
allNormalTextArray = allNormalText.split('\n')
len(allNormalTextArray)

185

In [4]:
from gensim.models import Word2Vec
from model import AdPredictor
adp = AdPredictor()
splitNormalWords = [list(adp.splitWords(ad, False)) for ad in allNormalTextArray]
splitNormalWords[0:10]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Kenvix\AppData\Local\Temp\jieba.cache
Loading model cost 0.789 seconds.
Prefix dict has been built successfully.


[['没有', '上次', '学长', '学姐', '教', '几节课'],
 ['学姐', '交', '一下'],
 ['认识', '学姐', '保研'],
 ['党明', '学姐', '真的', '强'],
 ['想不到', '没有', '党明', '学姐', '做', '不到'],
 ['党明', '学姐'],
 ['党明', '学姐', '献上'],
 ['学姐'],
 ['我见', '证明', '党明', '学姐', '非常', '漂亮', '女孩子'],
 ['开机', '慢点', '还好']]

In [5]:
from gensim import corpora
# 赋给语料库中每个词(不重复的词)一个整数id
dictionary = corpora.Dictionary(splitNormalWords)
new_corpus = [dictionary.doc2bow(text) for text in splitNormalWords]
print(dictionary.get("学姐"))
print(new_corpus[0:10])

None
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)], [(2, 1), (6, 1), (7, 1)], [(2, 1), (8, 1), (9, 1)], [(2, 1), (10, 1), (11, 1), (12, 1)], [(2, 1), (5, 1), (10, 1), (13, 1), (14, 1), (15, 1)], [(2, 1), (10, 1)], [(2, 1), (10, 1), (16, 1)], [(2, 1)], [(2, 1), (10, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)], [(22, 1), (23, 1), (24, 1)]]


In [6]:
from gensim import models
tfidf = models.TfidfModel(new_corpus)
tfidf.save("tfidf.gsm")

# 使用这个训练好的模型得到单词的tfidf值
tfidf_vec = []
tfidf_vocabs = []
for i in range(len(splitNormalWords)):
    string_bow = dictionary.doc2bow(splitNormalWords[i])
    string_tfidf = tfidf[string_bow]
    tfidf_vec.append(string_tfidf)

print(tfidf_vec[0:5])
tfidf_vocabs = dictionary.token2id
print(tfidf_vocabs.get("学姐"))
print(tfidf_vocabs.get("大佬"))
# print(tfidf_vocabs)
tfidf_vocabs = {value:key for (key, value) in tfidf_vocabs.items()}
print(tfidf_vocabs[0])

[[(0, 0.45687151407039367), (1, 0.45687151407039367), (2, 0.25535545406191246), (3, 0.45687151407039367), (4, 0.45687151407039367), (5, 0.3160178327324882)], [(2, 0.44996618130543437), (6, 0.556860388752772), (7, 0.6981668447581255)], [(2, 0.38899765237849326), (8, 0.6959786587088174), (9, 0.6035681677042755)], [(2, 0.3504975809135041), (10, 0.43376197428471686), (11, 0.6270959085570886), (12, 0.543831515185876)], [(2, 0.3016974387477618), (5, 0.3733688441637675), (10, 0.3733688441637675), (13, 0.42618820404090113), (14, 0.3964418831672419), (15, 0.5397846939992531)]]
2
153
上次


In [15]:
from collections import OrderedDict
import numpy as np

def tfidf_wtd_avg_word_vectors(words,tfidf_vector,tfidf_vocabulary,model,num_features):
    #print("tfidf_vector", tfidf_vector)
    word_tfidfs = []
    word_tfidf_map = OrderedDict()

    for (key, proba) in tfidf_vector:
        word_tfidfs.append(proba)
        word_tfidf_map[tfidf_vocabulary[key]] = proba
    
    # print(word_tfidfs)
    # print(word_tfidf_map)
    
    feature_vector=np.zeros((num_features,),dtype='float64')
    vocabulary=set(model.wv.index2word)
    wts=0
    for word in words:
        if word in vocabulary:
            word_vector=model[word]
            weighted_word_vector=word_tfidf_map[word]*word_vector
            wts=wts+word_tfidf_map[word]
            feature_vector=np.add(feature_vector,weighted_word_vector)
    if wts:
        feature_vector=np.divide(feature_vector,wts)
    return feature_vector

def tfidf_weighted_averaged_word_vectorizer(corpus,tfidf_vectors,tfidf_vocabulary,model,num_features):
    docs_tfidfs=[(doc,doc_tfidf) for doc,doc_tfidf in zip(corpus,tfidf_vectors)]
    features=[tfidf_wtd_avg_word_vectors(tokenized_sentence,tfidf,tfidf_vocabulary,model,num_features) for tokenized_sentence,tfidf in docs_tfidfs]
    return np.array(features)

# vec = Word2Vec(splitNormalWords,size=10,window=10,min_count=2,sample=1e-3)
word2vec = Word2Vec(splitNormalWords, min_count=3, sample=1e-3, size=10, window=10)
vected = tfidf_weighted_averaged_word_vectorizer(corpus=splitNormalWords, tfidf_vectors=tfidf_vec, tfidf_vocabulary=tfidf_vocabs,model=word2vec,num_features=10)
# vec.build_vocab(splitNormalWords)
# vec.train(splitNormalWords, total_examples=len(splitNormalWords), epochs=vec.epochs)
# # vec.save("vec.gsm")
# vec
print(len(vected))
vected

185


array([[ 0.02519699, -0.00593107,  0.00915863, ...,  0.01200001,
         0.00016478, -0.0022715 ],
       [ 0.0032771 , -0.03874716,  0.01412292, ..., -0.0296816 ,
         0.00575662, -0.03965287],
       [ 0.04038353, -0.04079113, -0.01122506, ..., -0.01659398,
        -0.03109673, -0.02944126],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.03878694,  0.04019498,  0.01958669, ...,  0.04764091,
        -0.02161361,  0.03267131],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])